# Lab CMBLenses

## Model

In [11]:
] activate "/home/jaimerz/julia_repos/CMBLensingInferenceTestProblem"

  Activating project at `~/julia_repos/CMBLensingInferenceTestProblem`


In [12]:
using CMBLensing, CMBLensingInferenceTestProblem, CUDA, LaTeXStrings, MCMCDiagnosticTools, Plots, ProgressMeter, Random, Zygote
ENV["LINES"] = 10;

In [13]:
using Revise
using MCHMC

In [ ]:
prob = load_cmb_lensing_problem(storage = Array,
                                T = Float32,
                                Nside = 32);

MAP_joint:  90%|████████████████████████████████████    |  ETA: 0:00:02
  step:        18
  logpdf:      24627.28
  α:           0.2220308
  CG:          2 iterations
  Linesearch:  15 bisections

In [ ]:
target = CMBLensingTarget(prob)

## Sample

In [ ]:
Ω = prob.Ωstart

In [ ]:
eps=0.005
L=sqrt(2)
spl=Sampler(;eps=eps, L=L)

In [ ]:
samples = Sample(spl, target; num_steps=10000, initial_x=Ω);

## Plotting

In [15]:
Plots.default(fmt=:png, dpi=120, size=(500,300), legendfontsize=10)

In [16]:
plot([exp(Ω.θ.r) for (Ω,) in chain], label=L"r", xlabel="step")
plot!([exp(Ω.θ.Aϕ) for (Ω,) in chain], label=L"A_\phi")

LoadError: UndefVarError: `chain` not defined

In [17]:
plot(
    histogram([exp(Ω.θ.r) for (Ω,) in chain], xlabel=L"r", label=nothing, lw=1),
    histogram([exp(Ω.θ.Aϕ) for (Ω,) in chain], xlabel=L"A_\phi", label=nothing, lw=1)
)

LoadError: UndefVarError: `chain` not defined

In [18]:
plot(
    histogram([exp(Ω.θ.r) for (Ω,) in chain], xlabel=L"r", label=nothing, lw=1),
    histogram([exp(Ω.θ.Aϕ) for (Ω,) in chain], xlabel=L"A_\phi", label=nothing, lw=1)
)

LoadError: UndefVarError: `chain` not defined

In [19]:
plot(
    histogram([exp(Ω.θ.r) for (Ω,) in chain], xlabel=L"r", label=nothing, lw=1),
    histogram([exp(Ω.θ.Aϕ) for (Ω,) in chain], xlabel=L"A_\phi", label=nothing, lw=1)
)

LoadError: UndefVarError: `chain` not defined

In [20]:

ps = map([(:ϕ°,:I,L"L",L"\phi^\circ"), (:f°,:E,"L\ell",L"E^\circ"), (:f°,:B,L"\ell",L"B^\circ")]) do (k1, k2, xlabel, title)
    plot(get_Cℓ(prob.Ωtrue[k1][k2]); label="true", xlabel, title)
    plot!(get_Cℓ(prob.Ωstart[k1][k2]); label="start", xlabel, title)
    plot!(get_Cℓ(chain[end][1][k1][k2]); label="last sample", xlabel, title)
end
plot(ps..., layout=(1,3), xscale=:log10, yscale=:log10, size=(1000,300), legend=:bottomleft)

LoadError: UndefVarError: `prob` not defined